## Assignment

In this exercise you will work with log data from the Xerces cloud service that is running in the Ericsson Research Data Center (ER DC) in Lund. The goal is to introduce you to the WARA-Ops portal and learn how to work with large data sets from a real-world cloud service.

### Introduction

Running and operating large-scale systems is a highly complex task. Data centers and cloud systems consist of a large number of moving parts. They are designed to be fault-tolerant and survive that components, such as disks, servers, and switch, occasionally break. This is typically handled through redundancy, i.e., using multiple identical subsystems.

Data-driven operations are a set of ideas and tools that combine concepts from several fields with the purpose of creating systems that self-manage. The area is becoming essential as modern computing and communication systems rapidly grow in size and complexity; managing these systems by hand is simply no longer possible.

However, the systems still break and require manual intervention. Typically this involves scanning through vast amounts of log-files to find out what is causing the outage and identify the root-cause. In the WARA-Ops project we aim at developing technology to do this automatically using for example machine learning.

The logs you will be looking were collected over approximately two weeks in the summer of 2023. Something broke around the first of June. We still don't what went wrong. If you find out, please let us known. It's very high volumes of data. The system emits 1200 log items per second.

In this assignment we want you to learn how work with real-world data sets, which are typically large, noisy, and undocumented. You should learn how to explore the data and find correlations, anomalies, etc., by for example visualising it or reducing dimensions.

### ERDC

ER DC and Xerces is operated by a team of IT professionals and adheres to strict Ericsson security and safety policies. This is important to protect potentially sensitive user data. We have been up and running since 2014 and serve many hundreds of researchers.

The entire data hall was designed for 2MW, but present, only 300-400kW is consumed (imagine the electricty bill!). The hardware setup on Xerces is approximetly

- 300 servers (20+cores, 256+GB RAM)
- 16 Nvidia V100 GPUs
- 48 Nvidia A100 GPUs
- 36 Nvidia GTX 1080 Ti GPUs
- 2 PB storage (spinning disc)
- 50 TB NVMe storage
- Dell 100 Gbps network fabric
- External Internet 2x10 Gbps

#### OpenStack

OpenStack is an open-source cloud computing platform that enables businesses and organizations to deploy and manage their own clouds. It is freely available and can be customized to fit the specific needs of an organization. OpenStack is designed to be scalable and flexible, allowing users to control large pools of compute, storage, and networking resources throughout a datacenter, all managed through a dashboard that gives administrators control while empowering users to provision resources through a web interface.

OpenStack consists of a set of independent subsystems or core components. The logs item that you will explore are emitted from theses subsystem. Sometimes you might want to look at the system as a whole and sometimes you want to zoom in on a particular component. Below is an incomplete list of the modules.

1. **Nova (Compute)**: Nova is the part of OpenStack that's responsible for managing and automating the pool of compute resources. These resources can be virtual machines or container instances. Nova interacts with the underlying virtualization technologies to manage the lifecycle of compute instances within OpenStack.

2. **Swift (Object Storage)**: Swift provides a scalable object storage system that can handle large amounts of unstructured data at a high level of availability. It's designed to store and retrieve any amount of data from anywhere – websites, mobile applications, IoT sensors, etc. It's highly durable and available, ensuring data is always accessible when needed.

3. **Cinder (Block Storage)**: Cinder offers persistent block storage to running instances. Its role is akin to that of a traditional hard drive, providing storage to virtual machines but with the flexibility and convenience that comes with virtualized environments. You can think of Cinder as providing the "hard drives" for your virtual machines.

4. **Neutron (Networking)**: Neutron delivers "networking-as-a-service" capabilities in OpenStack. It allows users to create and attach interfaces to networks and provides the API for users to define network connectivity and addressing in the cloud.

5. **Horizon (Dashboard)**: Horizon is the graphical interface for OpenStack. It's a web-based dashboard that allows both administrators and users to manage and provision resources within OpenStack. Through Horizon, users can interact with OpenStack's various services without needing to use the command line.

6. **Keystone (Identity Service)**: Keystone provides identity services for OpenStack. It's responsible for user management, authentication, and authorization. Essentially, Keystone allows you to control who can do what in OpenStack.

7. **Glance (Image Service)**: Glance allows users to discover, register, and retrieve virtual machine images. You can think of it as a repository or library of images that can be used to boot your virtual machines or instances.

8. **Heat (Orchestration)**: Heat is a service to orchestrate multiple composite cloud applications using templates, through both an OpenStack-native REST API and a CloudFormation-compatible Query API.

These are just some of the core components that make up OpenStack, each playing a crucial role in providing a comprehensive cloud computing platform. OpenStack is highly modular, meaning you can often add or replace components to suit your specific needs, making it a highly versatile choice for cloud infrastructure.

## Using WARA-Ops data and resources

WARA-Ops is a portal consisting of two parts. The data catalogue holds the data set. In this assignment you should be using the data set "ERDClogs-parsed", which consists of log data collected from the Xerces cloud service during the period 2023-05-21 -- 2023-06-03. The second part of the portal is a JupyterHub notebook compute service. You will find a link to the JupyterHub in the WARA-Ops portal. Please remember to generate a token so that you can access the data.
The two parts of the portal are connected using the WARA-Ops DataportalClient. The guide below will show you how to use it.

### Intro to WARA-Ops, ERDC and the computing environment

Since you are reading this, the assumption is that you have already logged into the WARA-Ops portal and managed to start Jupyter. If Jupyter is new to you, take a minute to check out the [documentation][1].

Much of the heavy lifting will be done by Pandas, and it is recommended that you keep [Pandas' documentation][2] handliy available in a browser window of its own.

We'll start this excercise with a warm-up, followed by an assignment that should be solvable given the warm-up examples, time, and grit.

[1]: https://docs.jupyter.org/en/latest/
[2]: https://pandas.pydata.org/pandas-docs/stable/index.html

#### Warm-up

The first thing to do, is to import Pandas and the client used to access the portal data:

In [ ]:
import pandas as pd

from dataportal import DataportalClient

Next, lets's create a couple of handy functions that will let us visualize data. For this we will use [matplotlib][1], sometimes with the aid of [seaborn][2].

[1]: https://matplotlib.org
[2]: https://seaborn.pydata.org/index.html

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def lineplot(list_x, list_y, title='', xlabel='', ylabel=''):
    """ Plot a line y of x

    Given a list of x- and y-values, plot a line.
    Optionally, provide title, xlabel and ylabel
    """
    plt.figure()
    plt.plot(list_x, list_y)
    plt.grid()

    _annotate_figure(title, xlabel, ylabel)
    plt.show()


def heatmap(df, title='', xlabel='', ylabel=''):
    """ Plot a 2-dimensional "heatmap" given a suitable Pandas' dataframe.

    Given a suitable Pandas' dataframe, plot a 2-dimensional "heatmap".
    Optionally, provide title, xlabel and ylabel
    """
    w = len(str(max(df.max()))) * len(df.columns.values) * 0.3
    h = len(df) * 0.3
    plt.figure(figsize=(w, h))

    sns.heatmap(df, annot=True, cmap='coolwarm', fmt='g')

    _annotate_figure(title, xlabel, ylabel)
    plt.show()


def _annotate_figure(title, xlabel, ylabel):
    plt.xticks(rotation=45, ha='right', va='top')
    plt.yticks(rotation=0)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

Let's give the helper functions a quick try, first `lineplot`:

In [ ]:
lineplot(range(1,100), range(1,100), title='Test',  xlabel='x-axis', ylabel='y-axis')

then create a "suitable" dataframe:

In [ ]:
test_df = pd.DataFrame(
    {
        "A": [1, 2, 3],
        "B": [4, 5, 6],
        "C": [7, 8, 9]
    }
)
test_df

that is fed to `heatmap`:

In [ ]:
heatmap(test_df)

Take a moment to recognize the features of the dataframe in the heatmap figure. Now would probably be a good time to skim the documentation [matplotlib][1] and [seaborn][2].


[1]: https://matplotlib.org
[2]: https://seaborn.pydata.org/index.html

Next, we'll interact with the data-portal, using the `DataportalClient` API. To do so you need the personal `token` from the portal.

Paste the token into the cell below, it should end up looking something like:

```.py
    # paste your token from the portal inside the quotes
    token = 'iOiJIUzI1NiIsInR5cCI6IkpXVCJ9.i4iLCJjbj13YXJhb3BzLXVzZXIsY249Z3JvdXBzLGNuPWF4iLCJjbj13YXJhb3BzLXVzZXIsY249Z3JvdXBzLGNuPWF'
    dataset = 'ERDClogs-parsed'
    client = DataportalClient(token)
```

Use `help(DataportalClient)` in a python cell to see some documentation.

In [ ]:
# paste your token from the portal inside the quotes
token = ''
dataset = 'ERDClogs-parsed'
client = DataportalClient(token)

After executing the above cell, it should print `Connection OK`, which means that the client is authenticated with the system.

However, before we can do anything we need to associate the client with a particular dataset:

In [ ]:
client.fromDataset(dataset)

Using the instantiated and configured client, we can examine the files in the dataset:

In [ ]:
file_iterator = client.listFiles()

Despite its name, `listFiles()` won't return a list of files, but an iteratable object so to use it we have to iterate over it (or force it to a list using `file_list = list(file_iterator)`)

In [ ]:
for file_info in file_iterator:
    print(file_info['FileID'])

We could pick any of the files, but we'll choose the file with FileID 36666:

In [ ]:
df = client.getData(fileID=36666) # load file content into a dataframe

Using `df.info()` provides some hints to the dataframe contents…

In [ ]:
df.info()

… as does `df.head()`.

In [ ]:
df.head()

Although `df.info()` lists most columns as being of type "object", `df.head()` suggests that those objects are in fact strings (they are). The columns `domain_id`,	`user_domain`, and `project_domain` are "de-indentified", containing just a single dash (-), and are thus of no interest to us.

Different OpenStack modules (see OpenStack section above) annotate log messages slightly different, but we can easily convert them to a "canonical" format by replacing e.g. `fatal`, `Fatal`, and `FATAL` with `FATAL` and so on for the other types:

In [ ]:
dict_change = {
    'fat': 'FATAL',
    'err': 'ERROR',
    'war': 'WARN',
    'inf': 'INFO',
    'deb': 'DEBUG',
    'tra': 'TRACE'
}

# Ex 1:
def fcn(entry):
    # If possible, unify log_level. Defaults to no change.
    if pd.notna(entry):
        return  dict_change.get(entry[:3].lower(), entry)
    return entry

df['log_level'] = df['log_level'].apply(fcn)

# Ex 2 (if you prefer a one-liner):
# df['log_level'] = df['log_level'].apply(lambda x: dict_change.get(x[:3].lower(), x) if pd.notna(x) else x)

Notice how the above code avoided iterating over the entries by using `apply` to _apply_ the same function to a whole column. If you find yourself wanting to loop over the elements of a column, you probably shouldn't…

We can e.g. use [`crosstab`][1] to count the number of log entries with a certain ID per [OpenStack module][2] (Nova, Neutron, …):

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html#pandas-crosstab
[2]: https://www.openstack.org/software/project-navigator/openstack-components#openstack-services

In [ ]:
# NOTE: crosstab is not able to use columns with NaN entries,
#       but `Logger` and `id` does not contain NaN.

# Count messages for each message id and Logger
df_count = pd.crosstab(index=df['id'], columns=df['Logger'])
# Display result as a heatmap using the functions we prepared above
heatmap(df_count, title='Number of times log with "id" is produced by module "Logger"', xlabel='Module', ylabel='message id')

It becomes clear from the heatmap that the network-as-a-service module Neutron produces most of the log messages, and messages with id 3385, 3083, and 3084 are the most frequent (in fact, they are _only_ produced by Neutron). Let's take a closer look at them:

In [ ]:
# return rows with id=3385, 3083, or 3084 AND columns log_level, id, and Payload but only unique messages:
df.loc[df['id'].isin([3385, 3083, 3084])][['log_level', 'id', 'Payload']].drop_duplicates()

Hmm, looks like 3385 is the same warning repeated 20000+ times (the boy who cried wolf…), let's see:

In [ ]:
df.loc[df['id'] == 3385][['log_level', 'id', 'Payload']].drop_duplicates()

Yup.

The messages 3083 and 3084 is the equivalent of a town criers' "It's 12 o'clock and all is well!'":

In [ ]:
df.loc[df['id'].isin([3083, 3084])][['log_level', 'id', 'Payload']].drop_duplicates()

----
**Assignment 1**:

Neither 3085 nor 3083/3084 are particularly helpful (or interesting) so for an exercise, create a heatmap like the one above but _without_ id 3385, 3083, and 3084 included.

_Hint: Check out pandas documentation for [isin][1] and the not operator (~)._

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html#pandas.DataFrame.isin

In [ ]:
# Your code here...

----

To visualize data over time, we need to make some preparations:

In [ ]:
# convert timestamp to datetime and set datetime as index
if df.index.name != '@timestamp':
    df.loc[:, ['@timestamp']] = pd.to_datetime(df['@timestamp'])
    df.set_index('@timestamp', inplace=True)

By this, data is [resampled][1] (to get _fewer_ points to plot) such that each sample corresponds to 5 seconds of data:

[1]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html#pandas.DataFrame.resample

In [ ]:
# Create samples. (This examples uses 5 seconds per sample).
samples = df.resample('5s') # use s, min, H, D

# Iterating over samples and count messages for each logger
# version 1:
list_counts = []
for start_datetime, group in samples:
    counts = group['Logger'].value_counts()
    counts.name = start_datetime
    list_counts.append(counts)
df_counts = pd.DataFrame(list_counts).fillna(0).astype(int)

# version 2: (same thing, but a little convoluted…)
# list_counts = [group['Logger'].value_counts().rename(start_datetime) for start_datetime, group in samples]
# df_counts = pd.DataFrame(list_counts).fillna(0).astype(int)


# now we can plot a heatmap
heatmap(df_counts, title='Count of log messages per module per 5s interval', xlabel='Module', ylabel='Time interval')

By chopping the data in timed intervals, we can create plots showing the rate (events/s) of logging in the OpenStack system.

First, use `groupby` to create a list of dataframes, each corresponding to the requested time interval:

In [ ]:
dfs = [g for g in df.groupby(pd.Grouper(freq='s'))]
len(dfs)

----
**Q:** If you google for e.g. "chunk pandas dataframe by time interval", you'll frequently find answers like:

`dfs = [g for g in df.groupby(df.index.second)]`

Why are those answers wrong (i.e. not the operation you expected)?

----

With the data grouped by time interval, we can plot the velocity:

In [ ]:
# Assumes dfs[0][1].index.name == '@timestamp'
time = []
velocity = []
for datetime, subframe in dfs:
    dt_start = subframe.index[0]
    dt_end = subframe.index[-1]
    velocity.append(len(subframe)/(dt_end - dt_start).total_seconds())
    time.append(datetime)

lineplot(time, velocity, title='Average velocity (events/s)')

----
**Assignment 2**:

Plot the log velocity like the one above but _without_ id 3385, 3083, and 3084 included.

In [ ]:
# Your code here...

----

### Handling large datasets

As a dataset is often in excess of 1GB, it can become a real problem to handle it in a limited amount of memory (RAM), as VMs are often limited to 8GB or 16GB, since working with a dataset expends it, creates copies of data, etc. In short, no matter how much memory you have access to, sooner or later you'll want to analyze data that is too big to fit in memory.

While the problem seems unsolvable, all is not lost. The WARA-ops portal (and client) lets you access data in "chunks". So far, you have only accessed the first 100 000 entries in the file, but we'll shortly fix that.

This is what `help(DataportalClient)` states:

    getData(fileID, entries=100000, generator=False)
        retrieves data from a file in the selected dataset as a pandas DataFrame

By default, a dataframe of 100 000 (default value of `entries`) rows is returned, but if `generator` is set to `True` it will instead return an object that can be iterated over to obtain a sequence of dataframes, each one consisting of `entries` rows. An example illustrates how to use it:

In [ ]:
dfg = client.getData(fileID=36666, generator=True)
for df in dfg:
    print(f"Rows {df.index[0]} to {df.index[-1]}")
    if df.index[-1] > 100000:
        break

First of all, the `break` statement is just a way to keep the output from the example short, in general you'd want the complete dataset. Secondly, even though the client and the docs mention _generator_, what is returned from `getData(fileID=fileID, generator=True)` is really an _iterable_ object, a minor quirk that sometimes causes confusion.

----
**Assignment 3**:

Now, evaluate a full logfile and produce a heatmap and a velocity plot but exclude all INFO events and messages with id 3385.
In other words, your task is to stich together a sequence of dataframes into a single (much larger) dataframe that contains only the columns necessary for plotting, with rows representing INFO events and messages with id 3385 filtered out.

In [ ]:
# Your code here...

----